# Лабораторная работа 3. Обучение без учителя

Результат лабораторной работы − отчет. Мы предпочитаем принимать отчеты в формате ноутбуков IPython (ipynb-файл). Постарайтесь сделать ваш отчет интересным рассказом, последовательно отвечающим на вопросы из заданий. Помимо ответов на вопросы, в отчете также должен быть код, однако чем меньше кода, тем лучше всем: нам − меньше проверять, вам — проще найти ошибку или дополнить эксперимент. При проверке оценивается четкость ответов на вопросы, аккуратность отчета и кода.

## Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 15 баллов. Сдавать задание после указанного срока сдачи нельзя. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов и понижают карму. Если вы нашли решение какого-то из заданий в открытом источнике, необходимо прислать ссылку на этот источник (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, нам необходима ссылка на источник).

## EM-алгоритм

### Краудсорсинг

Разметка данных — одна из самых трудозатратных задач в машинном обучении. Краудсорсинг позволяет распределить эту задачу на тысячи исполнителей, каждый из которых подготавливает небольшую часть датасета (подробнее <a href="https://academy.yandex.ru/posts/chto-takoe-kraudsorsing-i-pochemu-emu-nuzhno-uchitsya">тут</a>).

Пользователи могут допускать ошибки при разметке, кроме того, среди пользователей могут быть боты. Если мы попросим разметить каждый объект только одного пользователя, то с большой вероятностью получим не достаточно качественную разметку. Обычно каждый объект размечают несколько пользователей.

Результаты разметки нужно обработать. Самый простой метод *голос большинства*. Он заключается в том, что для каждого объекта нужно взять тот класс, который чаще всего ставили пользователи данному объекту. Это достаточно хороший метод, но он не учитывает различные особенности пользователей. Далее рассмотрим метод, который позволяет оценивать вероятность того, что разметчик ошибся.

### Метод Дэвида-Скина (Dawid, Skene, 1979)

Мы имеем в качестве данных $n_{ik}^u$ &mdash; количество раз, при которых разметчик $u \in U$ поставил класс $k \in K$ объекту $i \in I$ (возможно, разметчик видел этот объект несколько раз). Обозначим $Y_{ik} = I\{\text{объект $i$ класса $k$}\}$, это наши латентные величины. 

В качестве параметров имеем
* $\pi_{k\ell}^u$ &mdash; вероятность того, что разметчик $u$ поставил класс $\ell$ вместо правильного класса $k$. 
* $\rho_k$ &mdash; вероятность класса $k$.

Поймём, какой будет функция неполного правдоподобия в этой задаче. Прежде всего,

$$p_{\pi,p}(N, Y) = \prod_{i\in I}p(N_i, Y_i),$$

Если $k$ - номер класса $i$-го объекта, то

$$p(N_i, Y_i)=\underbrace{p(\text{объект $i$ класса $k$})}_{=\rho_k}p(N_i\mid\text{объект $i$ класса $k$})$$

(значения $Y_{it}$ однозначно определяются номером истинного класса, поэтому справа $Y_i$ пропадает). Далее, мы считаем, что разметчики действуют независимо, поэтому

$$p(N_i\mid\text{объект $i$ класса $k$}) = \prod_{u\in U}p(N_i^u\mid\text{объект $i$ класса $k$}).$$

Разберёмся с величиной $p(N_i^u\mid\text{объект $i$ класса $k$})$. Она отвечает за то, какие классы $u$-й разметчик ставил $i$-му объекту. Мы считаем, что встречи разметчика с объектом упорядочены по времени, тогда

$$p(\text{$u$-й разметчик отнёс $i$-й объект к классам $k'_1,\ldots,k'_r$}\mid\text{объект $i$ класса $k$}) =$$

$$=\prod_{s}p(\text{в $s$-ю встречу с $i$-м объектом $u$-й разметчик отнёс его к классу $k'_s$}\mid\text{объект $i$ класса $k$})$$

Эту вероятность можно переписать в виде

$$\prod_{\ell \in K} \left( \pi_{k\ell}^u \right)^{n_{i\ell}^u},$$

а итоговое неполное правдоподобие предстаёт в виде

$$p_{\pi,p}(N, Y) = \prod_{i\in I}\prod_{k \in K} \left( \rho_k \prod_{u\in U} \prod_{\ell \in K} \left( \pi_{k\ell}^u \right)^{n_{i\ell}^u} \right)^{Y_{ik}}$$

Его нам нужно максимизировать по $\pi$ и $\rho$

**Пояснение к формуле:** 

Вне больших скобок фиксируются объект и его класс, сама скобка возводится в степень 1, если рассматривается правильный класс объекта, и в степень 0 иначе. Внутри сначала записана вероятность того, что объект имеет данный класс, а затем &mdash; перебор по всем пользователям и всем классам, которые мог поставить данный пользователь. Наконец, записывается вероятность того, что пользователь нашему объекту поставил некоторый класс, которая возводится в степень того, сколько раз он поставил этот класс. Например, если пользователь видел изображение котика 5 раз, при этом 3 раза он сказал, что котик, а два раза &mdash; песик, то вероятность $\pi_{cat,cat}^u$ для данного котика учтется 3 раза, а вероятность $\pi_{cat,dog}^u$ &mdash; 2 раза.

**Задание 1 (2 балл)**

Распишите итерационную процедуру EM-алгоритма и значение ELBO в методе Дэвида-Скина.

---

__Шаг М.__ В выражении неполного правдоподобия нам известны $n_{il}^{(u)}, Y_{ik}$, максимизируя по $\pi$, получаем оценку:
$$\hat{\pi}_{kl}^{(u)}=\sum_{i}Y_{ik}n_{il}^{u} / \sum_{l}\sum_{i}Y_{ik}n_{ik}^{(u)}$$
При этом есть оценка на вероятность $p_k$:
$$\hat{p_k}=\sum_i Y_{ik} / I$$

__Шаг Е.__ Воспользуемся теоремой Байеса, чтобы получить оценки для $Y_{ik}$. Априори $p(Y_{ik}=1)=p_k$, имеем:
$$p(Y_{ik}=1|данные) \sim p(данные|Y_{ik}=1)p(Y_{ik}=1) \sim \prod_{u=1}^{U}\prod_{l=1}^{K}(\pi_{kl}^{(u)})^{n_{il}^{(u)}}p_k$$
Откуда получаем:
$$p(Y_{ik}=1|данные) = \prod_{u=1}^{U}\prod_{l=1}^{K}(\pi_{kl}^{(u)})^{n_{il}^{(u)}}p_k \big/ \sum_{q=1}^J\prod_{u=1}^{U}\prod_{l=1}^{K}(\pi_{ql}^{(u)})^{n_{il}^{(u)}}p_q$$

__ELBO (Evidence Lower BOund).__
$$\text{ELBO} = \mathbb{E}_q \log \frac{p_{\theta\phi}(t,w)}{q(t)} = \sum_{d=1}^{D}\sum_{i=1}^{n_d}\sum_{j=1}^{K}\mathbb{E}_q\mathbb{I}\{Y_{d_i}=j\}(\log\phi_{w_{d_i j} + \log \theta_{jd}}) - \sum_{d=1}^{D}\sum_{i=1}^{n_d}\sum_{j=1}^{K} \gamma_{jw_{d_i}d}\log\gamma_{jw_{d_i}d}$$

---

**Задание 2 (3 балла)** 

Реализуйте следующие методы агрегации результатов разметки в краудсорсинге:

* голосование по большинству;
* метод Дэвида-Скина.

Оба метода должны возвращать вероятность принадлежности объекта каждому из классов (итоговая метка получается выбором класса с наибольшей оценкой вероятности). 

Заметим, что метод голосования по большинству можно реализовать с помощью одной агрегирующей функцией `pandas.crosstab`, а метод Дэвида-Скина основывается на EM-алгоритме (при реализации стоит учесть, что EM-алгоритм сходится в локальные оптимумы, то есть его стоит запускать из разных начальных приближений).

Примените эти два метода к датасетам *Toloka Aggregation Relevance 2* и *Toloka Aggregation Relevance 5*, которые можно скачать <a href="https://toloka.ai/ru/datasets">тут</a>, и сравните их между собой. 

Обратите внимание, что в последнем датасете 5 различных меток, причем некоторые объекты в датасете имеют не один, а несколько правильных ответов. Любой из таких ответов считается правильным.

In [1]:
from google.colab import drive 
drive.mount('./drive')

Drive already mounted at ./drive; to attempt to forcibly remount, call drive.mount("./drive", force_remount=True).


In [2]:
import pandas as pd

path = './drive/MyDrive/Colab Notebooks/ML-2/Lab 3/'
kwargs = {'sep': '\t', 'header': None}

agg2_crowd = pd.read_csv(path + 'TlkAgg2/crowd_labels.tsv', names=['toloker', 'object', 'label'], **kwargs)
agg2_golden = pd.read_csv(path + 'TlkAgg2/golden_labels.tsv', names=['object', 'label'], **kwargs)
agg5_crowd = pd.read_csv(path + 'TlkAgg5/crowd_labels.tsv', names=['toloker', 'object', 'label'], **kwargs)
agg5_golden = pd.read_csv(path + 'TlkAgg5/golden_labels.tsv', names=['object', 'label'], **kwargs)
agg5_bans = pd.read_csv(path + 'TlkAgg5/bans.tsv', names=['toloker', 'ban'], **kwargs)

print(agg2_crowd.shape, agg2_golden.shape, agg5_crowd.shape, agg5_golden.shape, agg5_bans.shape)

(475536, 3) (10079, 2) (1091918, 3) (46146, 2) (90, 2)


In [3]:
agg2_crowd.head(2)

,toloker,object,label
0,w851,t30685,1
1,w6991,t30008,0


In [4]:
agg2_golden.head(2)

,object,label
0,t30006,0
1,t33578,0


Реализуем голосование большинством:

In [5]:
def majority_voting(crowdsourced: pd.DataFrame) -> pd.DataFrame:
    res = pd.crosstab(index=crowdsourced['object'], 
                      columns=crowdsourced['label'], 
                      normalize='index').reset_index()
    res.columns.name = ''
    return res

In [6]:
agg2_crowd_voted = majority_voting(agg2_crowd)
agg2_crowd_voted.head(3)

,object,0,1
0,t0,0.4,0.6
1,t1,0.0,1.0
2,t10,0.2,0.8


Реализуем метод Дэвида-Скина:

In [61]:
# TODO

**Задание 3 (1 балл)** 

Попробуйте в методе Дэвида-Скина в качестве начального приближения вероятностей классов для каждого объекта подавать те вероятности, которые посчитаны методом голосования по большинству, и провести сначала M-шаг.

In [ ]:
# TODO

Далее мы будем работать с таблицей `vacancies.csv` и нам нужно подготовить данные для работы, а именно сделайте следующее:
    
- Разбейте данные на обучающую выборку (строки, не содержащие метки кластеров) и тестовую (строки, содержащие метки кластеров)
- Предобработайте текст, содержащийся в колонках *name* и *description* (уберите артефакты, нормализуйте и т.д.)

In [8]:
# установка pymystem3
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /root/.local/bin/mystem

--2022-04-15 14:30:07--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.241, 5.45.205.244, 5.45.205.243, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.241|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cachev2-spb01.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz?lid=122 [following]
--2022-04-15 14:30:08--  https://cachev2-spb01.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz?lid=122
Resolving cachev2-spb01.cdn.yandex.net (cachev2-spb01.cdn.yandex.net)... 37.140.137.1, 2a02:6b8:0:2221::301
Connecting to cachev2-spb01.cdn.yandex.net (cachev2-spb01.cdn.yandex.net)|37.140.137.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz.1’

mystem-3.0-linux3.1 100%[

In [41]:
import numba
import nltk
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
import re

nltk.download("stopwords")
m = Mystem()
stop_words = stopwords.words("russian")
html_tags = re.compile('<.*?>')

# @numba.njit
def cleanhtml(raw_html):
  cleantext = re.sub(html_tags, '', raw_html)
  return cleantext

# @numba.njit
def preprocess(text: str) -> str:
    text = cleanhtml(text)
    tokens = m.lemmatize(text.lower())
    tokens_filtered = []
    for token in tokens:
        if (token not in stop_words) and (token != " ") and (token.strip() not in punctuation):
            tokens_filtered.append(token)
    text = " ".join(tokens_filtered)
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [42]:
vacancies = pd.read_csv(path + 'vacancies.csv')
vacancies['description'][0]

'<strong>Обязанности:</strong> <ul> <li>Устранение повреждений на абонентских линиях связи по технологиям Fttb, xDSL, традиционной телефонии, SIP телефонии, GRON.</li> <li>Выполнение инсталляций услуг.</li> </ul> <strong>Требования:</strong> <ul> <li>Среднее профессиональное (техническое) образование.</li> <li>Опыт работы с линейными сооружениями связи, ремонта и обслуживания ПК и локальных сетей.</li> </ul> <strong>Условия:</strong> <ul> <li> <p>Работа в крупнейшем федеральном телеком-операторе России;</p> </li> <li> <p>Белая заработная плата;</p> </li> <li> <p>Система бонусов по итогам работы за месяц;</p> </li> <li> <p>Оформление по ТК РФ (официальная заработная плата, оплачиваемый больничный).</p> </li> <li> <p>Место работы г. Тула.</p> </li> <li> <p>График работы нормированный сменный 2/2 с 9:00 до 21:00..</p> </li> </ul>'

In [43]:
preprocess(vacancies['description'][0])

'обязанность устранение повреждение абонентский линия связь технология fttb xdsl традиционный телефония sip телефония gron выполнение инсталляция услуга требование средний профессиональный технический образование опыт работа линейный сооружение связь ремонт обслуживание пк локальный сеть условие работа крупный федеральный телеком оператор россия белый заработный плата система бонус итог работа месяц оформление тк рф официальный заработный плата оплачивать больничный место работа г тула график работа нормировать сменный 2 2 9 00 21 00 ..'

In [ ]:
## Too long :(
%%time
vacancies['name'] = vacancies['name'].apply(preprocess)
vacancies['description'] = vacancies['description'].apply(preprocess)

In [44]:
train = vacancies.loc[~vacancies['category'].isna()]
test = vacancies.loc[vacancies['category'].isna()]

In [45]:
train.head(1)

,id,name,description,key_skills,schedule_id,employment_id,experience_id,accept_handicapped,accept_kids,salary_from,salary_to,salary_gross,salary_currency,area_id,employer_id,type_id,created_at,published_at,category
10,10,Инженер-конструктор 1 категории,<strong>Обязанности:</strong> <ul> <li>Подгото...,NaN,fullDay,full,between3And6,False,False,60000.0,80000.0,False,RUR,1,1047669.0,open,2019-09-11T17:52:11+0300,2019-09-11T17:52:11+0300,9.0


### Тематическое моделирование

Тематическое моделирование заключается в поиске тем $T$, которые хорошо бы описывали документы $D$ со словарём $W$. Большинство тематических моделей оперирует данными в формате "мешка слов", т.е. учитывают только частоты слов в документах, а не их порядок. Одной из простейших тематических моделей является [PLSA](https://en.wikipedia.org/wiki/Probabilistic_latent_semantic_analysis), которая приводит к задаче стохастического матричного разложения: 

$$F \approx \Phi \times \Theta$$
где
- $F_{W \times D}$— матрица распределений слов в документах (нормированные частоты)
- $\Phi_{W \times T}$ — матрица распределений слов в темах (модель)
- $\Theta_{T \times D}$ — матрица распределений тем в документах (результат применения модели к обучающим данным)

Можно сказать, что алгоритмы тематического моделирования производят мягкую бикластеризацию данных:
 - *мягкую*, так как объекты относятся не строго к одному кластеру, а к нескольким с разными вероятностями
 - *бикластеризацию*, так как модель одновременно кластеризует слова по темам и темы по документам.
 
 С вероятностной точки зрения, задача обучения модели PLSA ставится как максимизация неполного правдоподобия по параметам $\Phi$ и $\Theta$. ЕМ-алгоритм для модели PLSA заключается в повторении двух шагов:

- **Е-шаг** — оценка распределений тем для каждого слова в каждом документе по параметрам $\Phi$ и $\Theta$ (шаг 6);
- **М-шаг** — обновление параметров $\Phi$ и $\Theta$ на основе полученных оценок (шаги 7 и 9).

Существуют различные модификации итерационного процесса, позволяющие снизить расходы по памяти. В данном случае, мы избежим хранения трехмерной матрицы $p_{tdw}$, сразу пересчитывая $\Theta$ для текущего документа и аккумулируя счетчики $n_{wt}$ для последующего пересчета $\Phi$.

Псевдокод алгоритма записывается следующим образом:

1. Инициализировать $\phi_{wt}^0$ для всех $w \in W$, $t \in T$ и $\theta_{td}^0$ для всех $t \in T$, $d \in D$
2. Внешний цикл по итерациям $i = 1 ... max\_iter$:
3. $\quad$ $n_{wt}^i := 0$, $n_t^i := 0$ для всех $w \in W$ и $t \in T$ 
4. $\quad$ Внутренний цикл по документам $d \in D$  
5. $\qquad$ $Z_w := \sum_{t \in T} \phi_{wt}^{i-1}\theta_{td}^{i-1}$ для всех $w \in d$ $\cfrac{}{}$
6. $\qquad$ $p_{tdw} := \cfrac{ \phi_{wt}^{i-1}\theta_{td}^{i-1} }{ Z_w }$ (**E-шаг**)
7. $\qquad$ $\theta_{td}^{i} := \cfrac{ \sum_{w \in d} n_{dw} p_{tdw} }{ n_d }$ для всех $t \in T$ (**M-шаг**)
8. $\qquad$ Увеличить $n_{wt}^i$ и $n_t^i$ на $n_{dw} p_{tdw}$ для всех $w \in W$ и $t \in T$
9. $\quad \phi_{wt}^i := \cfrac{n_{wt}^i}{n_t^i}$ для всех $w \in W$ и $t \in T$ (**M-шаг**)

Обозначения:
 - $p_{tdw}$ — вероятность темы $t$ для слова $w$ в документе $d$
 - $\phi_{wt}$ — элемент матрицы $\Phi$, соответствующий вероятности слова $w$ в теме $t$
 - $\theta_{td}$ — элемент матрицы $\Theta$, соответствующий вероятности темы $t$ в документе $d$
 - $n_{wt}$ — элемент матрицы счётчиков отнесения слова $w$ к теме $t$ (путем нормирования этой матрицы получается матрица $\Phi$)
 - $Z_w$ — элемент вектора вспомогательных переменных, соответствующий слову $w$
 - $n_t$ — вектор нормировочных констант для матрицы $n_{wt}$
 - $n_d$ — вектор нормировочных констант для матрицы $n_{dw}$
 - $n$ — суммарное число слов в коллекции

Для оценивания качества построенной модели и контроля сходимости процесса обучения обычно используют [перплексию](http://www.machinelearning.ru/wiki/images/8/88/Voron-iip9-talk.pdf):

$$\mathcal{P} = \exp\bigg(- \frac{\mathcal{L}}{n} \bigg) = \exp\bigg(- \cfrac{1}{n}\sum_{d \in D}\sum_{w \in d} n_{dw} \ln \big(\sum_{t \in T}\phi_{wt}\theta_{td} \big)\bigg)$$

Это традиционная мера качества в тематическом моделировании, которая основана на правдоподобии модели $\mathcal{L}$. Число итераций $max\_iter$ в алгоритме обучения следует выбирать достаточным для того, чтобы перплексия перестала существенно убывать. Однако известно, что перплексия плохо отражает интерпретируемость найденных тем, поэтому помимо нее обычно используются дополнительные меры или экспертные оценки.

**Рекомендации к реализации:**

- При делении на нулевые значения нужно просто заменить частное на ноль.
- ЕМ-алгоритм стоит реализовывать с использованием векторных операций. Для проверки корректности реализации сперва можно написать скалярную версию, после чего векторизовать её, удостоверившись, что обе реализации дают одинаковый результат. Невекторизованный алгоритм может работать в сотни раз медленнее векторизованного, и его использование может привести к невозможности выполнения задания.
- Итерационный процесс следует начинать, инициализировав матрицы $\Phi$ и $\Theta$. Инициализация может быть случайной, важно не забыть отнормировать столбцы матриц.
- Неэффективная реализация перплексии может в разы замедлить работу алгоритма.

**Задание 4 (3 балла)** 

Реализуйте описанный выше ЕМ-алгоритм для модели *PLSA* и добавьте в вашу реализацию подсчёт перплексии. 

Примените ваш алгоритм к подготовленным ранее данным (объедините текст из колонок *name* и *description*), рассмотрев число тем T = 5, а также:

* Постройте график значения перплексии в зависимости от итерации (убедитесь в корректности реализации: график перплексии должен быть невозрастающим). 
* Выведите для каждой темы топ-20 наиболее вероятных слов.

Посмотрите внимательно на полученные темы. Как вам кажется, получились ли они интерпретируемыми?

In [ ]:
# TODO

**Задание 5 (1 балл)** 

Рассмотрите большее число тем (10, 20) и несколько различных начальных приближений. Проанализируйте результаты и ответьте на следующие вопросы: 

- Mожно ли сказать, что интерпретируемость каждой темы изменяется с ростом их числа?
- Устойчив ли алгоритм к начальному приближению на примере идентичности топовых слов в соответствующих темах?
- Отражает ли перплексия качество получаемых моделей? В чём заключается причина хорошего/плохого соответствия?

In [ ]:
# TODO

## Кластеризация

**Задание 6 (2 балла)** 

В данном задание следуют сравнить между собой алгоритмы [KMeans](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html), [DBSCAN](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html) и [Birch](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.Birch.html) используя подготовленные ранее данные.

Поэксперементируйте с различными способами векторизации текста, например можно:

- использовать только *name* / *description*, объединить их в один текст или сконкатенировать векторные представления,
- использовать представление в виде мешка слов или какую-либо его модификацию,
- сократить размерность векторного представления, используя PCA или векторные представления слов,

или сделать что-то более интересное, что вы придумаете!

Выберете лучшую комбинацию векторизации и алгоритма кластеризации и визуализируйте полученные кластеры (например, воспользовавшись облаком тегов, или предложите свой способ). Обоснуйте почему вы считаете, что выбранный вами подход для решения задачи кластеризации вакансий является лучшим.

**Задание "со звездочкой" (1 балл)** 

Попробуйте обогатить векторное представление полученное из текстов другими признаками из таблицы `vacancies.csv` и добиться лучшей интерпретируемости кластеров.

In [53]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans, DBSCAN, Birch
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer

n_clusters = train['category'].nunique()
print('n_clusters =', n_clusters)

vectorizer = CountVectorizer()
corpus = train['name'] + train['description']
X = vectorizer.fit_transform(corpus)

n_clusters = 27


In [ ]:
reduced_data = PCA(n_components=2).fit_transform(X.toarray())
kmeans = KMeans(init="k-means++", n_clusters=n_clusters, n_init=4)
kmeans.fit(reduced_data)

Визуализация из [доки](https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_digits.html) - Out of memory :(

In [ ]:
# Step size of the mesh. Decrease to increase the quality of the VQ.
h = 0.02  # point in the mesh [x_min, x_max]x[y_min, y_max].

# Plot the decision boundary. For that, we will assign a color to each
x_min, x_max = reduced_data[:, 0].min() - 1, reduced_data[:, 0].max() + 1
y_min, y_max = reduced_data[:, 1].min() - 1, reduced_data[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

# Obtain labels for each point in mesh. Use last trained model.
Z = kmeans.predict(np.c_[xx.ravel(), yy.ravel()])

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure(1)
plt.clf()
plt.imshow(
    Z,
    interpolation="nearest",
    extent=(xx.min(), xx.max(), yy.min(), yy.max()),
    cmap=plt.cm.Paired,
    aspect="auto",
    origin="lower",
)

plt.plot(reduced_data[:, 0], reduced_data[:, 1], "k.", markersize=2)
# Plot the centroids as a white X
centroids = kmeans.cluster_centers_
plt.scatter(
    centroids[:, 0],
    centroids[:, 1],
    marker="x",
    s=169,
    linewidths=3,
    color="w",
    zorder=10,
)
plt.title(
    "K-means clustering on the digits dataset (PCA-reduced data)\n"
    "Centroids are marked with white cross"
)
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.xticks(())
plt.yticks(())
plt.show()

## Частичное обучение

Часто у нас есть размеченная выборка только для небольшой части выборки. Тогда мы можем применить подходы _частичного обучения (semi-supervised learning)_. Более подробно про реализацию таких методов в sklearn можно прочитать в разделе [semi-supervised](http://scikit-learn.org/stable/modules/label_propagation.html#semi-supervised).

**Задание 7 (2 балла)** 

В этом задание нужно сделать следующее: 

- Разделите объекты, у которых существуют метки, на обучающую и тестовую выборки (при этом не обязательно делить в соотношении 70% на 30%). Обогатите обучающую выборку объектами без меток.
- Воспользовавшись опытом выполнения задания 6, возьмите "лучшее" векторное представление вакансий и обучите [LabelSpreading](http://scikit-learn.org/stable/modules/generated/sklearn.semi_supervised.LabelSpreading.html) (подберите лучшие параметры, опираясь на [F1 score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)).
- Попробуйте запустить алгоритм несколько раз, отмечая известными различные объекты, а также меняя пропорции разбиения, посчитайте качество и визуализируйте результаты. Можно ли сказать что алгоритм сильно зависит от известных начальных объектов? Есть ли класс, для которого это больше всего заметно?

In [ ]:
# TODO